In [1]:
import os, shutil

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error

import pandas as pd
import numpy as np

# To plot pretty figures
import matplotlib as mpl
import matplotlib.pyplot as plt

import scipy.stats as stats

import seaborn as sns

from datetime import datetime

import re

from tqdm.notebook import tqdm

import dask.dataframe as dd

from typing import List, Dict

In [ ]:
client = Client()
client

# Merge

In [2]:
#Rutas de los archivos
ruta_2019 = '/Users/sandradiaz/Documents/CaptstoneProject/dades/data/2019_BicingNou_ESTACIONS_MOD/*.csv'
ruta_2021 = '/Users/sandradiaz/Documents/CaptstoneProject/dades/data/2021_BicingNou_ESTACIONS_MOD/*.csv'
ruta_2022 = '/Users/sandradiaz/Documents/CaptstoneProject/dades/data/2022_BicingNou_ESTACIONS_MOD/*.csv'

#Leer archivos CSV en dataframes de Dask
data_2019 = dd.read_csv(ruta_2019)
data_2021 = dd.read_csv(ruta_2021)
data_2022 = dd.read_csv(ruta_2022)

#Merge (full outer join)
merged_data = dd.merge(data_2019, data_2021, how='outer')
merged_data = dd.merge(merged_data, data_2022, how='outer')

#Resultado
#bbdd_completa = merged_data.compute()
bbdd_completa = merged_data

In [3]:
bbdd_completa.columns

Index(['station_id', 'num_docks_available', 'num_bikes_available', 'status',
       'timestamp', 'is_installed', 'is_renting', 'is_returning',
       'is_charging_station', 'num_bikes_available_types.mechanical',
       'num_bikes_available_types.ebike', 'year', 'month', 'dayofweek', 'day',
       'dayofyear', 'hour', 'capacity', 'ctx0', 'ctx1', 'ctx2', 'ctx3',
       'ctx4'],
      dtype='object')

In [4]:
bbdd_completa.head(2)

,station_id,num_docks_available,num_bikes_available,status,timestamp,is_installed,is_renting,is_returning,is_charging_station,num_bikes_available_types.mechanical,...,dayofweek,day,dayofyear,hour,capacity,ctx0,ctx1,ctx2,ctx3,ctx4
0,1,4.0,24.0,1,1546916400,1,1,1,1,24.0,...,1,8,8,3,46.0,0.086957,0.086957,0.112319,0.12253,0.146245
1,1,24.0,6.0,1,1547974800,1,1,1,1,6.0,...,6,20,20,9,46.0,0.521739,0.521739,0.521739,0.51087,0.543478


In [5]:
%%time
bbdd_completa.shape[0].compute(),bbdd_completa.shape[1]

CPU times: user 1min 51s, sys: 1min 28s, total: 3min 20s
Wall time: 2min 39s


12531377

# Unique Station ID

In [7]:
unique_ids_by_year = bbdd_completa.groupby('year')['station_id'].unique().compute()
print(unique_ids_by_year)

year
2019    [1, 320, 319, 318, 317, 316, 315, 314, 313, 31...
2020    [23, 247, 384, 129, 75, 238, 380, 480, 58, 50,...
2021    [1, 271, 400, 510, 26, 270, 269, 401, 268, 267...
2022    [1, 380, 28, 379, 378, 377, 376, 375, 29, 374,...
Name: station_id, dtype: object


In [8]:
%%time
# Calcula los station_id comunes en todos los años
common_ids = set(unique_ids_by_year[2019])
for ids in unique_ids_by_year:
    common_ids = common_ids.intersection(set(ids))
common_ids

CPU times: user 343 µs, sys: 98 µs, total: 441 µs
Wall time: 560 µs


{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 53,
 54,
 55,
 56,
 57,
 58,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 92,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 170,
 171,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194

In [10]:
filtered_bbdd = bbdd_completa[bbdd_completa['station_id'].isin(common_ids)]

In [11]:
%%time
filtered_bbdd.shape[0].compute(),filtered_bbdd.shape[1] 
#bbdd_completa.shape = (12531377, 23)

CPU times: user 1min 52s, sys: 1min 37s, total: 3min 30s
Wall time: 2min 56s


(11594600, 23)

## IDs no comunes (testing)

In [47]:
nonfiltered_bbdd = bbdd_completa[~bbdd_completa['station_id'].isin(common_ids)]

# Data exploration__________________________________

## · Descriptiva
Media, mediana, desviación estándar, mínimo y máximo para columnas numéricas Idea general de la distribución y el rango de los valores en esas columnas.

In [24]:
def calculate_statistics(data, variables):
    statistics = {}
    
    for variable in variables:
        # Calcular estadísticas para cada variable
        mean = data[variable].mean()
        median = data[variable].median()
        std = data[variable].std()
        minimum = data[variable].min()
        maximum = data[variable].max()
        
        # Guardar las estadísticas en un diccionario
        statistics[variable] = {
            'Mean': mean,
            'Median': median,
            'Standard Deviation': std,
            'Minimum': minimum,
            'Maximum': maximum
        }
    
    return statistics

# Variables para calcular las estadísticas
variables = ['num_docks_available', 'num_bikes_available', 'capacity']

# Carga la base de datos como un DataFrame de Dask
#bbdd_completa = dd.read_csv('datos.csv')  # Reemplaza 'datos.csv' con la ubicación y nombre de tu archivo CSV

# Llama a la función y pasa el DataFrame y las variables como argumentos
result = calculate_statistics(bbdd_completa, variables)

# Print
for variable, stats in result.items():
    print(f"Variable: {variable}")
    for stat, value in stats.items():
        print(f"{stat}: {value}")
    print()


Variable: num_docks_available
Mean: 15.79022992082025
Median: 16.166666666666668
Standard Deviation: 8.62290193896031
Minimum: 0.0
Maximum: 54.0

Variable: num_bikes_available
Mean: 9.532674617904897
Median: 7.8
Standard Deviation: 8.242537498715903
Minimum: -0.1666666666666666
Maximum: 54.0

Variable: capacity
Mean: 27.007916765830593
Median: 27.000000000000007
Standard Deviation: 6.166117481650443
Minimum: 0.0
Maximum: 54.00000000000003



## · Visualización de la disponibilidad de bicicletas
Gráficos para visualizar la disponibilidad de bicicletas a lo largo del tiempo. Ejemplo: cantidad de bicicletas disponibles en función del tiempo para identificar patrones diarios, semanales o estacionales.

## · Análisis temporal
Datos por año, mes, día de la semana o hora del día para conocer estadísticas agregadas, como el promedio de bicicletas disponibles o el porcentaje de anclajes disponibles. Tendencias temporales y patrones de uso.

## · Relación entre variables
Explorar si hay una correlación entre el número de anclajes disponibles (num_docks_available) y el número de bicicletas disponibles (num_bikes_available). Investigar si la disponibilidad de bicicletas difiere entre los diferentes tipos de bicicletas (mecánicas y eléctricas).

## · Análisis de capacidad y porcentaje de anclajes disponibles
Capacidad (capacity) y los porcentajes de anclajes disponibles (ctx0, ctx1, ctx2, ctx3, ctx4). Promedio distribución de la capacidad y los porcentajes de anclajes disponibles. Objetivo: entender la utilización de las estaciones de bicicletas a lo largo del tiempo.

# Predicción__________________________________
a) Regresión lineal: relación lineal entre las variables de entrada y la variable de salida. 

b) Regresión Redes Neuronales (RNN -redes neuronales recurrentes-): pueden capturar relaciones no lineales entre las variables de entrada y salida.

# Datos adicionales
Tiempo, festivos Barcelona